<a href="https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/ComfyUI_digiclau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이 Google Colab notebook은 [ComfyUI](https://github.com/comfyanonymous/ComfyUI)를 실행해 줍니다.

## ❗❗❗ [ComfyUI Google Colab 사용 설명 영상](https://youtu.be/HUY2WdbNQmc) - 자세한 사용 설명을 보실수 있습니다.
### ❗❗❗ 이 Colab notebook을 사용하기 전에, 바로 위에 있는 사용 설명 영상을 먼저 시청하시기 바랍니다.

### [DigiClau Youtube](https://www.youtube.com/channel/UCllY989CI5F-eU1BJ-rwEvw) - 저의 채널도 방문해 주세요.

### 복사본을 사용할 경우, 원본은 [이곳](https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/ComfyUI_digiclau.ipynb)에 있습니다.

### 🍷🍷🍷 [SD Automatic1111 Colab도 있습니다.](https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/StableDiffusionWebUI_digiclau.ipynb) 🍷🍷🍷
### 🍷🍷🍷 [Stable Diffusion WebUI Forge Colab도 있습니다.](https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/StableDiffusionWebUI_digiclau_Forge.ipynb) 🍷🍷🍷

## [☕커피 사주세요](https://www.buymeacoffee.com/digiclau)

## ### 최근 업데이트 정보
- 2024년 9월:
 - CloudFlare 문제 해결
- 2024년 6월:
 - ComfyUI에 필요한 Pytorch 및 dependency 업데이트


In [ ]:
#@title <font size="6" color="black">ComfyUI 설치 및 실행 (MyDrive/ComfyUI)</font>
from pathlib import Path

# === 사용자 설정 옵션 ===
USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}

# === Google Drive 마운트 및 경로 설정 ===
if USE_GOOGLE_DRIVE:
    print("📂 Google Drive 마운트 중...")
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_DIR = "/content/drive/MyDrive/ComfyUI"
else:
    BASE_DIR = "/content/ComfyUI"

COMFYUI_DIR = BASE_DIR
Path(BASE_DIR).mkdir(parents=True, exist_ok=True)

# === ComfyUI 설치 ===
%cd {str(Path(COMFYUI_DIR).parent)}
if not Path(COMFYUI_DIR).exists():
    print("🛠️ ComfyUI 최초 설치 중...")
    !git clone https://github.com/comfyanonymous/ComfyUI

# === ComfyUI 디렉토리로 이동 및 업데이트 ===
%cd {COMFYUI_DIR}
if UPDATE_COMFY_UI:
    print("🔄 ComfyUI 업데이트 중...")
    !git pull

# === 의존성 설치 ===
print("📦 의존성 설치 중...")
!pip install -r requirements.txt
!pip install accelerate torchsde einops safetensors aiohttp pyyaml Pillow scipy tqdm psutil sentencepiece kornia soundfile transformers>=4.28.1 tokenizers>=0.13.3
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# === ComfyUI-Manager 설치 및 업데이트 ===
if USE_COMFYUI_MANAGER:
    MANAGER_DIR = Path(COMFYUI_DIR) / "custom_nodes" / "ComfyUI-Manager"
    Path(MANAGER_DIR.parent).mkdir(exist_ok=True)

    %cd {MANAGER_DIR.parent}
    if not MANAGER_DIR.exists():
        print("🧩 ComfyUI-Manager 설치 중...")
        !git clone https://github.com/ltdrdata/ComfyUI-Manager
    %cd {MANAGER_DIR}
    !git pull

# === 커스텀 노드 의존성 설치 ===
if INSTALL_CUSTOM_NODES_DEPENDENCIES:
    print("🔧 커스텀 노드 의존성 복원 중...")
    !pip install GitPython
    !python {MANAGER_DIR}/cm-cli.py restore-dependencies

#@title <font size="6" color="black">Run ComfyUI with cloudflared (Recommended Way)</font>
##################################################
# Run ComfyUI with cloudflared (Recommended Way) #
##################################################
# === cloudflared 설치 및 ComfyUI 실행 ===
print("🌐 cloudflared 설치 중...")
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket

def start_cloudflared(port=8188):
    def tunnel_thread():
        while True:
            time.sleep(0.5)
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            result = sock.connect_ex(('127.0.0.1', port))
            sock.close()
            if result == 0:
                break

        print("\n✅ ComfyUI 준비 완료. Cloudflared 터널 시작 중...\n")
        p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        for line in p.stderr:
            l = line.decode()
            if "trycloudflare.com " in l:
                print("🔗 ComfyUI 접근 URL:", l[l.find("http"):], end='')

    threading.Thread(target=tunnel_thread, daemon=True).start()

start_cloudflared()

# === ComfyUI 실행 ===
%cd {COMFYUI_DIR}
!python3 main.py --dont-print-server